### IMPORTS

In [1]:
# load, preprocess, scale, baseline
from wrangle import final_dataset

# manipulate data
import numpy as np

# visualize data
import matplotlib.pyplot as plt
import seaborn as sn

# model preparation
from sklearn.model_selection import (learning_curve,
                                     LearningCurveDisplay, 
                                     validation_curve,
                                     ValidationCurveDisplay)

# neural network weights
from models import good_weights

# optmization problem
from models import good_problem

np.random.seed(123)

### Load, Clean, Preprocess, Scale, Baseline: Cardiovascular Disease

In [2]:
X_train_scaled_cd, X_test_scaled_cd, y_train_cd, y_test_cd = final_dataset(dataset='cvd')

CVD Loading and Cleaning...
CVD Loaded and Cleaned...

CVD Splitting...
CVD Split...

CVD Scaling...
CVD Scaled...

Baseline Accuracy Score: 0.51%



### Modeling: Cardiovascular Disease

In [3]:
nn = good_weights(X_train_scaled_cd, y_train_cd,
                 X_test_scaled_cd, y_test_cd)

Running All Algorithms
Running Random Hill Climbing
Running Simulated Annealing
Running Genetic Algorithm
Running Gradient Descent
Successfully Run All Algorithms

Working on: Random Hill Climbing
Fitting: Random Hill Climbing


In [ ]:
nn['Random Hill Climbing']

### Optimization Problem: Cardiovascular Disease

In [ ]:
opt_prob = good_problem()

In [ ]:
opt_prob['Random Hill Climbing']

### Plotting: Cardiovascular Disease

In [ ]:
# Create/display fitness curve plot for selected algorithm.
# plt.plot(nn.fitness_curve)
# plt.ylabel('Relative Fitness Found')
# plt.show()